# Napari Visualization Pipeline

In [92]:
import napari
import numpy as np
from skimage import morphology, restoration, filters, measure
from tifffile import imread

img = imread('cell-data/parent_control_z3_ch00.tif')

viewer = napari.Viewer()
viewer.add_image(img, name='nuclei')

<Image layer 'nuclei' at 0x344100a10>

### Erosion

In [93]:
from skimage.morphology import erosion, disk

eroded = erosion(img, disk(1)) # Change to 2 or 3 for more intense erosion

viewer.add_image(eroded, name='eroded')

<Image layer 'eroded' at 0x343a3a3f0>

### Estimate noise and denoise via non-local means

In [94]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import img_as_float

img_float = img_as_float(eroded)

sigma_est = np.mean(estimate_sigma(img_float, channel_axis=None)) # Estimated noise standard deviation. Lower SD means lower estimated noise.

denoised = denoise_nl_means(
    img_float,
    h=0.8 * sigma_est, # h is filter strength. This line sets filter strength to 80% of estimated noise level, but tweak this number if necessary.
    patch_size=5, # Larger value = captures more context; better denoising but loses fine detail (probably doesn't matter for this use).
    patch_distance=6, # Larger value = searches wider area, so better denoising but slower processing.
    fast_mode=True # Make False for more accurate denoising (but it'll be a bit slower).
)

viewer.add_image(denoised, name='denoised')

<Image layer 'denoised' at 0x344103b30>

### Create nucleus mask

In [95]:
threshold = 0.1 # Play around with the threshold to find the optimal value.
thresholded = denoised > threshold # For each pixel --> if it's brighter than the threshold value, count it as foreground (part of a nucleus).
# The line above creates a boolean array.

viewer.add_labels(thresholded.astype(int), name='thresholded') # This turns the boolean array into binaries.

<Labels layer 'thresholded' at 0x3438d3da0>

### Fill any gaps in the mask regions

In [96]:
from scipy.ndimage import binary_fill_holes
import numpy as np

# Assuming 'mask' is a binary numpy array where nuclei are True (1) and background is False (0)
filled_mask = binary_fill_holes(thresholded)

# Convert back to int if needed
filled_mask = filled_mask.astype(np.uint8)

viewer.add_labels(filled_mask, name='hold-filled-mask')

<Labels layer 'hold-filled-mask' at 0x31521ca70>

### Separate mask into distinct regions with integer labels

In [97]:
from skimage.measure import label

labeled_mask = label(filled_mask) # Label connected regions of the mask

viewer.add_labels(labeled_mask, name='nuclei_mask')

<Labels layer 'nuclei_mask' at 0x376b52120>

### Clean mask by removing nucleus regions that are cut off by the image and by removing isolated pixels that remain after erosion and denoising

In [98]:
from skimage.measure import regionprops
import numpy as np

image_height, image_width = labeled_mask.shape
cleaned_mask = np.zeros_like(labeled_mask)

for region in regionprops(labeled_mask):
    min_row, min_column, max_row, max_column = region.bbox

    # This checks for labeled regions at the edges of the image
    touches_edge = (min_row == 0 or min_column == 0 or max_row == image_height or max_column == image_width)
    
    if not touches_edge and region.area >= 500:
        cleaned_mask[labeled_mask == region.label] = region.label 
        # This line sets all pixels in cleaned_mask at positions where the boolean mask is True to the integer value region.label.
        # Each specific integer value is a specific label/color.
        
viewer.add_labels(cleaned_mask, name='cleaned_mask')

<Labels layer 'cleaned_mask' at 0x37f6c8ef0>

### Determine the image's microns per pixel ratio and create the first donut mask

In [99]:
from skimage.morphology import dilation, disk
from skimage.segmentation import find_boundaries
import tifffile

with tifffile.TiffFile('cell-data/parent_control_z3_ch00.tif') as tif:
    page = tif.pages[0]
    tags = page.tags

    x_res = tags['XResolution'].value  # (num, denom)
    y_res = tags['YResolution'].value
    res_unit = tags['ResolutionUnit'].value  # 3 = centimeters

    # Convert to pixels per micrometer
    if res_unit == 3:  # Centimeters
        x_ppcm = x_res[0] / x_res[1]
        y_ppcm = y_res[0] / y_res[1]
        x_ppum = x_ppcm / 10000  # cm → µm
        y_ppum = y_ppcm / 10000
        print(f'X pixels/µm: {x_ppum:.4f}. Compare these values to the metadata in Fiji ImageJ.')
        print(f'Y pixels/µm: {y_ppum:.4f}. Compare these values to the metadata in Fiji ImageJ.')
    else:
        print('Unsupported resolution unit:', res_unit)

microns_per_pixel = (1 / x_ppum + 1 / y_ppum) / 2
desired_donut_width_um = 2
n_pixels = int(desired_donut_width_um / microns_per_pixel)

outer_mask = dilation(cleaned_mask, disk(n_pixels))

donut_mask = outer_mask - cleaned_mask
donut_mask[donut_mask < 0] = 0  # Just in case

donut_mask = donut_mask.astype(np.uint8)

viewer.add_labels(donut_mask, name='donut')

X pixels/µm: 5.5081. Compare these values to the metadata in Fiji ImageJ.
Y pixels/µm: 5.5081. Compare these values to the metadata in Fiji ImageJ.


<Labels layer 'donut' at 0x343728bf0>

## Notes:
- Make outer donut
- Delete masks whose donuts overlap 